# Project setting

In [1]:
!git clone https://github.com/fuzzythecat/awesome-spacer

Cloning into 'awesome-spacer'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 104 (delta 36), reused 38 (delta 14), pack-reused 27
Receiving objects: 100% (104/104), 1.48 MiB | 12.01 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [2]:
%cd awesome-spacer
!ls

/content/awesome-spacer
data  images  LICENSE  models  notebooks  README.md  train.py


In [3]:
!wget 'https://www.dropbox.com/s/vq00bl6pl6rp1d1/27-0.0359-0.9864.hdf5'
!ls

--2019-12-14 02:06:45--  https://www.dropbox.com/s/vq00bl6pl6rp1d1/27-0.0359-0.9864.hdf5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/vq00bl6pl6rp1d1/27-0.0359-0.9864.hdf5 [following]
--2019-12-14 02:06:45--  https://www.dropbox.com/s/raw/vq00bl6pl6rp1d1/27-0.0359-0.9864.hdf5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf48c76016f90b4b005251edee6.dl.dropboxusercontent.com/cd/0/inline/AuPXTEvsv1LM5w7Ya8QsTyr71qV6oyZ2DAY-Ol31e-hhBgsQJaUHjAJLpOiihXmE0eDS3lfWOa8dEB2JG2feC48tz1ew79B-Fcuc-VIsMQa5YzO_Vw_qZGF7GY6zel2WRyI/file# [following]
--2019-12-14 02:06:46--  https://ucf48c76016f90b4b005251edee6.dl.dropboxusercontent.com/cd/0/inline/AuPXTEvsv1LM5w7Ya8QsTyr71qV6oyZ2DAY-Ol31e-hhBgsQJaUHjAJLpOiihXmE0eDS3lfWOa8dEB2JG2feC48

In [4]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 42kB/s 
     |████████████████████████████████| 450kB 57.0MB/s 
     |████████████████████████████████| 3.8MB 45.4MB/s 
     |████████████████████████████████| 81kB 13.0MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.8.2 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goog

In [5]:
# Check that TensorFlow 2.0 is installed correctly.
import tensorflow as tf
tf.__version__

'2.0.0'

# Model configuration

In [0]:
import os
import argparse
import tensorflow as tf

from glob import glob
from random import shuffle
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences

from models.config import Config
from models.utils import encode_string
from models.spacer import Spacer

In [0]:
# In Colab / Jupyter Notebook / etc. it is hard to change the contents
# of config.py. Override configuration settings for custom inference settings. 
class InferenceConfig(Config):
    MAX_TEXT_LEN = 200
    EMBEDDING_DIM = 100
    FILTER_SIZES = [4, 6, 8, 10, 12]
    FILTER_NUMS = [256, 256, 256, 256, 256]
    
config = InferenceConfig()

In [0]:
model = Spacer('27-0.0359-0.9864.hdf5',
               config)

In [0]:
tests = [ 
    '''술에 포함된 알코올은 칼로리가 상당히 높은 편이고, 
몸에 알코올이 들어가면 신체의 대사를 방해해 지방 등 주요 영양소가 몸속에 쌓이게 돼 안주를 먹지 
않고 술만 마셔도 살찌는 것은 똑같다는 말입니다.''',

    '''오늘도 또 우리 수탉이 막 쫓기었다. 내가 점심을 먹고 나무를 하러 갈 양으로 나올 때이었다. 
산으로 올라서려니까 등뒤에서 푸드득 푸드득 하고 닭의 횃소리가 야단이다. 
깜짝 놀라서 고개를 돌려보니 아니나 다르랴 두 놈이 또 얼리었다. ''',

    '''손흥민의 활약에 전 세계가 들썩이고 있다. 
지난 주말 번리전에서 75미터를 폭발적인 스피드로 돌파해 득점을 한 후 전 세계가 손흥민의 
원더골에 찬사를 보내고 있다. 당시 득점을 보면 손흥민은 공을 잡고 주위를 살피며 
동료들에게 공을 넘겨주려고 했지만 공간이 보이지 않자, 본인이 빠르게 판단해 치고 올라갔다. 
여기에 고속 드리블을 시도하는 순간에도 계속 동료를 살피며 수비수들의 시선을 분산시켰고, 
결국 환상적인 득점을 만들었다.''',

    '''한국관광공사곽진광관광과장''',

    '''내가그린기린그림은긴기린그린그림이고, 
네가그린기린그림은길지않은기린그린그림이다.''',

    '''아무튼 동아시아에도 상투를 자르기 전에 내 목을 먼저 쳐라, 
예의의 나라가 어떻게 양이의 법도를 따르겠냐라며 엄청난 저항이 있었지만, 
결국 서구문명을 완전히 받아들이고 흡수했습니다.''',

    '''병원에 응급 환자가 실려 왔다. 클리블랜드에게 3쿼터 막판~4쿼터 초반 구간 0-24 런(RUN) 일방적인 
폭행을 당했던 휴스턴이다. 러셀 웨스트브룩, 클린트 카펠라 등 수련생들은 어찌할 바를 몰라
발을 동동 굴렀다. 여기서 전문의 털보 선생님 등장. "모두 물러나라. 내가 치료하겠다." 
환자는 '털보네이터'의 자상한 개인 전술 기반 연속 13득점 시술을 받은 후 기적적으로 소생했다. 
털보는 팀이 4쿼터 마지막 8분 구간에서 기록한 28득점 중 23득점을 생산한 후 유유히 퇴근한다.
(본인 20득점+AST 기반 동료 3점) 3점슛 10개 포함 55득점(FG 20/34)은 치료비다. ''',

    '''영국의철학자인화이트헤드는”서양의 2000년 철학은 모두 플라톤의 각주에 불과하다”라고 말했으며,
시인 에머슨은 “철학은 플라톤이고, 플라톤은 철학”이라 평하였는데, 플라톤은 소크라테스의 수제자이다. 
플라톤이 20대인 시절, 스승 소크라테스가 민주주의에 의해 끝내 사형당하는 것을 보고 크게 분개했으며, 
이는 그의 귀족주의 “철인 정치” 지지의 큰 계기가 되었다. 알렉산더 대왕은 소크라테스의 증손제자로, 
플라톤의 제자인 아리스토텔레스의''',

    '''프랑스는 지나치게 경직된 노동시장과 과도한 복지와 연금으로 인한 재정지출로 
성장 드라이브를 걸지 못하고 있었습니다만, 마크롱의 일부개혁을 통해 실업율은 
상대적으로 감소하였고, 또 성장율도 현재의 글로벌 경기침체에도 불구하고 상당히 선방하고 
있습니다. 마크롱은 좌파도, 우파도 아니라고 선언하면서 동시에 "성장"과 "혁신"을 강조했습니다. 
그리고 실제로 마크롱 정부는 벤처를 적극적으로 지원하였고, 역대급으로 많은 스타트업이 
생겼다고 합니다.'''
]

# Remove whitespaces
tests = [''.join(t.split()) for t in tests]
encoded = [encode_string(t, config.CHR_TO_IDX) for t in tests]
encoded = pad_sequences(encoded, maxlen=config.MAX_TEXT_LEN, 
                        padding='post', truncating='post',
                        value=config.CHR_TO_IDX['<PAD>'])

In [0]:
probs = model.predict(encoded)

In [25]:
for i, t in enumerate(tests):
    print('#{}: '.format(i+1))
    for j, c in enumerate(t[:config.MAX_TEXT_LEN]):
        print(c, end='')
        if probs[i][j] >= 0.5:
            print(' ', end='')
    print('\n'*2)

#1: 
술에 포함된 알코올은 칼로리가 상당히 높은 편이고, 몸에 알코올이 들어가면 신체의 대사를 방해해 지방 등 주요 영양소가 몸 속에 쌓이게 돼 안주를 먹지 않고 술만 마셔도 살찌는 것은 똑같다는 말입니다.


#2: 
오늘도 또 우리 수탉이 막 쫓기었다. 내가 점심을 먹고 나무를 하러 갈 양으로 나올 때이었다. 산으로 올라서려니까 등뒤에서 푸드득 푸드득 하고 닭의 횃소리가 야단이다. 깜짝 놀라서 고개를 돌려보니 아니나 다르랴 두 놈이 또 얼리었다.


#3: 
손흥민의 활약에 전세계가 들썩이고 있다. 지난 주말번리전에서 75미터를 폭발적인 스피드로 돌파해 득점을 한 후 전세계가 손흥민의 원더골에 찬사를 보내고 있다. 당시 득점을 보면 손흥민은 공을 잡고 주위를 살피며 동료들에게 공을 넘겨주려고 했지만 공간이 보이지 않자, 본인이 빠르게 판단해 치고 올라갔다. 여기에 고속드리블을 시도하는 순간에도 계속 동료를 살피며 수비수들의 시선을 분산시켰고, 결국 환상적인 득점을 만들었다.


#4: 
한국관광공사 곽진광 관광과장


#5: 
내가 그린 기린 그림은 긴 기린 그린 그림이고, 네가 그린 기린 그림은 길지 않은 기린 그린 그림이다.


#6: 
아무튼 동아시아에도 상투를 자르기 전에 내 목을 먼저 쳐라, 예의의 나라가 어떻게 양이의 법도를 따르겠냐라며 엄청난 저항이 있었지만, 결국 서구문명을 완전히 받아들이고 흡수했습니다.


#7: 
병원에 응급환자가 실려왔다. 클리블랜드에게 3쿼터 막판~4쿼터 초반 구간 0-24런(RUN) 일방적인 폭행을 당했던 휴스턴이다. 러셀 웨스트 브룩, 클린트 카펠라 등 수련생들은 어찌할 바를 몰라 발을 동동 굴렀다. 여기서 전문의 털보선생님 등장. "모두 물러나라. 내가 치료하겠다." 환자는 '털보네이터'의 자상한 개인전술기반 연속 13득점 시술을 받은 후 기적적으로 소생했다. 털보는 팀이 4쿼터 마지막 8분 구간에서 기록한 28득점 중 23득점을 생산한 


#8: 
경기 후 컨퍼런스 무리뉴 : 난 그런 기분이 들었어, 예전